# 0. Introduction to Prompt Engineering with Bedrock

This notebook will introduce you to prompt engineering concepts using Amazon Bedrock. We'll start with basic prompts and incrementally add structure and complexity.

## Setting up Bedrock client

Let's set up the Bedrock client using our utility functions:

In [ ]:
import sys
sys.path.append('./')

from src.utils import (
    create_bedrock_client,
    text_completion,
    extract_json_from_text,
    invoke_with_prefill,
    CLAUDE_3_5_SONNET,
    CLAUDE_3_5_HAIKU,
    NOVA_LITE
)

# Create a Bedrock client
bedrock_client = create_bedrock_client()

# Default settings
TEMPERATURE = 0.0  # Lower temperature = more deterministic outputs

## 1. Basic Prompt

Let's start with a very simple prompt to understand how the model responds to basic instructions.

In [ ]:
prompt = "What is prompt engineering?"

response = text_completion(bedrock_client, prompt, model_id=CLAUDE_3_5_HAIKU, temperature=TEMPERATURE)
print(response)

## 2. Defining the LLM's Role

Explicitly telling the LLM to adopt a specific role or persona helps shape its responses to match the expected expertise and tone.

In [ ]:
role_prompt = """
You are an expert in artificial intelligence and machine learning, specializing in large language models.

What is prompt engineering and why is it important for working with generative AI?
"""

response = text_completion(bedrock_client, role_prompt, model_id=CLAUDE_3_5_HAIKU, temperature=TEMPERATURE)
print(response)

## 3. Providing Reference Context

Adding specific information as context helps the model ground its responses in facts you provide rather than its own training data alone. This is essential for domain-specific tasks or when working with proprietary information.

First, let's define our context as a structured object, simulating data that might come from a database or API:

In [ ]:
# Define our context as a structured data object (could come from a database, API, etc.)
product_context = {
    "name": "ProductX",
    "type": "AI analytics platform",
    "features": [
        {"name": "Data Processing", "description": "Real-time processing up to 10TB/hour"},
        {"name": "Connectivity", "description": "Support for 50+ data connectors including AWS, GCP, and Azure"},
        {"name": "Visualization", "description": "Custom dashboard with drag-and-drop interface"},
        {"name": "AI Capabilities", "description": "Anomaly detection with 99.7% accuracy"},
        {"name": "Automation", "description": "Automated report generation and scheduling"}
    ],
    "pricing": [
        {"tier": "Basic", "cost": "$1,000/month", "users": 5, "storage": "1TB"},
        {"tier": "Professional", "cost": "$5,000/month", "users": 20, "storage": "10TB"},
        {"tier": "Enterprise", "cost": "Custom pricing", "users": "Unlimited", "storage": "Custom"}
    ]
}

In [ ]:
reference_context_prompt = f"""
# Product Context (JSON)
{product_context}

---

Using the above product context, explain the key features of ProductX to a potential enterprise customer who needs to process 8TB of data daily and has a team of 15 analysts.
"""

response = text_completion(bedrock_client, reference_context_prompt, model_id=CLAUDE_3_5_HAIKU, temperature=TEMPERATURE)
print(response)

## 4. Adding Structure: Instructions and Rules

Adding structure to your prompts using clear sections helps organize the information and makes it easier for the model to follow your requirements.

In [ ]:
structured_prompt = """
## Instructions
You are an expert in generative AI. Explain the concept of prompt engineering to beginners who are just starting with large language models.

## Rules
- Keep your explanation concise and use simple language
- Include 3-5 practical tips for effective prompt engineering
- Provide 2 example prompts: one poorly engineered and one well-engineered
- Explain why the well-engineered prompt would produce better results
"""

response = text_completion(bedrock_client, structured_prompt, model_id=CLAUDE_3_5_HAIKU, temperature=TEMPERATURE)
print(response)

## 5. Requesting Structured Output

You can ask the model to format its response in a specific way, such as JSON, bullet points, or tables.

In [ ]:
structured_output_prompt = """
## Instructions
You are an expert in generative AI. Explain prompt engineering techniques and best practices.

## Rules
- Return your response as a JSON object with the following structure:
  - "definition": A brief definition of prompt engineering
  - "importance": Why prompt engineering is important
  - "techniques": An array of objects, each with a "name" and "description" field for different prompt engineering techniques
  - "examples": An array of objects, each with a "good_prompt", "bad_prompt", and "explanation" field

- Return the JSON inside ```json code blocks
"""

json_response = text_completion(bedrock_client, structured_output_prompt, model_id=CLAUDE_3_5_HAIKU, temperature=TEMPERATURE)
print(json_response)

## 5.1. Extracting and Using the JSON Output

Let's extract the JSON from the response and use it in our code:

In [ ]:
# Extract and parse the JSON
try:
    json_data = extract_json_from_text(json_response)
    
    # Access specific fields
    print("Definition of prompt engineering:")
    print(json_data["definition"])
    print("\nPrompt engineering techniques:")
    for technique in json_data["techniques"]:
        print(f"- {technique['name']}: {technique['description']}")
except ValueError as e:
    print(f"Error extracting JSON: {e}")

## 6. Response Prefilling

Response prefilling lets you start the model's response with specific text, which can help ensure you get the format you want.

In [ ]:
prompt = """
## Instructions
You are an expert in generative AI. Create a cheat sheet for prompt engineering techniques.

## Rules
- Format your response as a JSON object
- Include techniques, each with a name, description, and example
"""

# Prefill the response to ensure we get the format we want - avoid trailing whitespace!
prefill = """```json
{
  "prompt_engineering_techniques": [
    {
      "name":"""

completion = invoke_with_prefill(bedrock_client, prompt, prefill, model_id=NOVA_LITE, temperature=TEMPERATURE)

# Combine the prefill and the completion for the full response
full_response = prefill + completion
print(full_response)

## 7. Zero-shot vs. Few-shot Learning

Let's compare zero-shot (no examples) with few-shot (providing examples) approaches for a classification task.

In [ ]:
# Zero-shot classification
zero_shot_prompt = """
## Instructions
Classify the following customer feedback as positive, negative, or neutral.

## Customer feedback
"The product arrived on time but was missing a few parts. Customer support was helpful and sent the missing pieces quickly."

## Rules
- Output only one word: positive, negative, or neutral
"""

response = text_completion(bedrock_client, zero_shot_prompt, model_id=CLAUDE_3_5_HAIKU, temperature=TEMPERATURE)
print("Zero-shot classification result:")
print(response)

In [ ]:
# Few-shot classification with examples
few_shot_prompt = """
## Instructions
Classify the following customer feedback as positive, negative, or neutral.

## Examples
Example 1:
"The service was terrible and staff was rude."
Classification: negative

Example 2:
"Amazing product! Exceeded my expectations in every way."
Classification: positive

Example 3:
"The product works as described. Nothing special but does its job."
Classification: neutral

## Customer feedback
"The product arrived on time but was missing a few parts. Customer support was helpful and sent the missing pieces quickly."

## Rules
- Output only one word: positive, negative, or neutral
"""

response = text_completion(bedrock_client, few_shot_prompt, model_id=CLAUDE_3_5_HAIKU, temperature=TEMPERATURE)
print("Few-shot classification result:")
print(response)

## 8. Advanced Prompt Engineering: Chain of Thought

Chain of Thought is a technique that encourages the model to break down complex reasoning into steps, which typically produces more accurate results for complex tasks.

In [ ]:
chain_of_thought_prompt = """
## Instructions
You are a problem-solving assistant. Solve the following word problem step-by-step.

## Problem
James has 5 boxes. Each box contains 8 books. He donates 12 books to the library. 
Then he buys 3 more boxes with 8 books each. How many books does James have now?

## Rules
- Think through this problem step by step
- Show your reasoning for each step
- After showing your work, provide the final answer
"""

response = text_completion(bedrock_client, chain_of_thought_prompt, model_id=CLAUDE_3_5_HAIKU, temperature=TEMPERATURE)
print(response)

## 9. Self-Verification

Self-verification is a powerful technique where you ask the model to verify its own output. This is especially useful for tasks like code generation or SQL queries where correctness is crucial.

In [ ]:
self_verification_prompt = """
## Instructions
You are a SQL expert. Convert the following natural language question into a SQL query for a database containing customer and order information.

## Database Schema
- customers(id, name, email, signup_date, country)
- orders(id, customer_id, order_date, total_amount, status)
- products(id, name, category, price)
- order_items(order_id, product_id, quantity)

## Question
What are the top 5 countries by total order value in the last 3 months?

## Rules
1. First, think step by step about how to solve this problem
2. Then, write the SQL query inside ```sql code blocks
3. After writing the query, verify your solution by:
   - Checking if it correctly addresses the original question
   - Ensuring all necessary joins are included
   - Confirming that date filtering is correctly applied
   - Making sure the aggregation and sorting are appropriate
4. If you find any issues during verification, update your query and explain the changes
"""

response = text_completion(bedrock_client, self_verification_prompt, model_id=CLAUDE_3_5_HAIKU, temperature=TEMPERATURE)
print(response)

In [ ]:
# Extract just the SQL query from the response
import re

def extract_sql_from_text(text):
    # Look for SQL code blocks
    match = re.search(r"```(?:sql)?\s*\n(.*?)```", text, re.DOTALL)
    
    if match:
        return match.group(1).strip()
    else:
        # If no code blocks, try to find SQL-like content directly
        match = re.search(r"SELECT.*?FROM.*?(?:WHERE|GROUP BY|ORDER BY|LIMIT|;)", text, re.DOTALL | re.IGNORECASE)
        if match:
            return match.group(0).strip()
        return "No SQL query found"

# Extract the SQL query
sql_query = extract_sql_from_text(response)
print("Extracted SQL Query:")
print("------------------")
print(sql_query)
print("------------------\n")

In [ ]:
# Simple example of how you might use this query in a real application
print("Example: Using the generated SQL query with a database connection")
print("""
import sqlite3
import pandas as pd
from datetime import datetime, timedelta

# Connect to your database
conn = sqlite3.connect('your_database.db')

# Execute the query
result_df = pd.read_sql_query(sql_query, conn)

# Display the results
print(result_df.head())

# Close the connection
conn.close()
""")

## 10. Controlling Response Length

You can instruct the model to provide responses of different lengths based on your requirements.

In [ ]:
length_control_prompt = """
## Instructions
Explain what artificial intelligence is at three different levels of detail.

## Rules
- First, give a one-sentence explanation (10-15 words)
- Then, give a short paragraph explanation (50-75 words)
- Finally, give a detailed explanation (150-200 words)
- Label each section: "One-sentence", "Short paragraph", and "Detailed explanation"
"""

response = text_completion(bedrock_client, length_control_prompt, model_id=CLAUDE_3_5_HAIKU, temperature=TEMPERATURE)
print(response)

## 11. Next Steps

In this notebook, we've explored several prompt engineering techniques:

1. Basic prompting
2. Defining the LLM's role
3. Providing reference context
4. Structuring prompts with instructions and rules
5. Requesting structured outputs (JSON)
6. Extracting and using JSON from responses
7. Response prefilling
8. Zero-shot vs. Few-shot learning
9. Chain of thought reasoning
10. Self-verification
11. Controlling response length

In the next notebook, we'll explore more advanced techniques like working with multimodal inputs (images and text).